In [1]:
import tensorly as tl
import numpy as np
from numpy import linalg as la
from sympy import *
from sympy import Matrix, symbols, solve_linear_system
from sympy.solvers.solveset import linsolve
import time
from joblib import Parallel, delayed
x,y,z_1,z_2,z_3 = symbols('x y z_1 z_2 z_3')
x,y,z1,z2,z3 ,b= symbols("x,y,z1,z2,z3,b")
p_112, p_102, p_002, p_012 = symbols('p_112 p_102 p_002 p_012')
p_000, p_001, p_100, p_101, p_200, p_201, p_010, p_011, p_020, p_021, p_110, p_111, p_210, p_211, p_120, p_121, p_220, p_221 = symbols('p_000 p_001 p_100 p_101 p_200 p_201 p_010 p_011 p_020 p_021 p_110 p_111 p_210 p_211 p_120 p_121 p_220 p_221')

In [2]:
from tensorly.decomposition import parafac
from tensorly.decomposition import non_negative_parafac_hals

def check_rank(tensor, rank, non_neg=True, n=10, tol=0.001, p=False):
    if non_neg:
        for k in range(n):
            weights, factors = non_negative_parafac_hals(tensor, n_iter_max=1000000, rank=rank, init='random')
            full = tl.cp_to_tensor((weights, factors))
            diff = (full - tensor) / tensor
            if p:
                # print(full)
                print(tl.max(abs(diff)))
            if tl.max(abs(diff)) < tol:
                return True
    else:
        for k in range(n):
            weights, factors = parafac(tensor, n_iter_max=1000000, rank=rank)
            full = tl.cp_to_tensor((weights, factors))
            diff = (full - tensor) / tensor
            if p:
                print(tl.max(abs(diff)))
            if tl.max(abs(diff)) < tol:
                return True
    return False

In [3]:
#generates random rank 3 tensors
def rank_tree():
    return (low_tensor() + low_tensor() + low_tensor()) 

# random tensor of some dim
def rand_tensor():
    return tl.tensor(np.random.randint(1, 100000, size=(3,2,2)))*1.0

# generates rank 1 tensors
def low_tensor():
    max = 300
    a = np.random.randint(1, max, size=3) 
    b = np.random.randint(1, max, size=2)
    c = np.random.randint(1, max, size=2)
    tens = tl.tensor(np.kron(np.kron(a, b), c).reshape(3, 2, 2)) * 1.0
    return tens

# checks if all subtensors have nonneg rank 3
def check(t):
    t1 = tl.tensor([t[0], t[1]])
    t2 = tl.tensor([t[1], t[2]])
    t3 = tl.tensor([t[0], t[2]])
    a1 = det(Matrix(t1[0]))
    a2 = det(Matrix(t1[1]))
    a3 = det(Matrix(t2[0]))
    a4 = det(Matrix(t2[1]))
    b1 = det(Matrix(t1[:,0]))
    b2 = det(Matrix(t1[:,1]))
    b3 = det(Matrix(t2[:,0]))
    b4 = det(Matrix(t2[:,1]))
    c1 = det(Matrix(t1[:,:,0]))
    c2 = det(Matrix(t1[:,:,1]))
    c3 = det(Matrix(t2[:,:,0]))
    c4 = det(Matrix(t2[:,:,1]))

    a5 = det(Matrix(t3[0]))
    a6 = det(Matrix(t3[1]))

    b5 = det(Matrix(t3[:,0]))
    b6 = det(Matrix(t3[:,1]))

    c5 = det(Matrix(t3[:,:,0]))
    c6 = det(Matrix(t3[:,:,1]))
    return sgn([a1, a2,a3,a4,a5,a6]) or sgn([b1,b2,b3,b4,b5,b6]) or sgn([c1,c2,c3,c4,c5,c6])

# determines sign of a list
def sgn(a):
    t = 0
    ab = 0
    for a_i in a:
        t+= abs(a_i)
        ab += a_i
    return t == abs(ab)


# checks if the give 2x2x2 tensor has nonneg rank 3
def check_r3(t):
    a1 = det(Matrix(t[0]))
    a2 = det(Matrix(t[1]))
    b1 = det(Matrix(t[:,0]))
    b2 = det(Matrix(t[:,1]))
    c1 = det(Matrix(t[:,:,0]))
    c2 = det(Matrix(t[:,:,1]))
    return sgn([a1,a2]) or sgn([b1,b2]) or sgn([c1,c2])

# checks if given 2x2x2 has nonneg rank 2
def check_r2(t):
    a1 = det(Matrix(t[0]))
    a2 = det(Matrix(t[1]))
    b1 = det(Matrix(t[:,0]))
    b2 = det(Matrix(t[:,1]))
    c1 = det(Matrix(t[:,:,0]))
    c2 = det(Matrix(t[:,:,1]))
    d1 = ineq(a1,a2,b1,b2,c1,c2,ge,ge,ge)
    d2 = ineq(a1,a2,b1,b2,c1,c2,le,le,ge)
    d3 = ineq(a1,a2,b1,b2,c1,c2,le,ge,le)
    d4 = ineq(a1,a2,b1,b2,c1,c2,ge,le,le)
    supermod = d1 or d2 or d3 or d4
    return supermod


# helpers
def ineq(a1,a2,b1,b2,c1,c2,f1,f2,f3):
    t1 = f1(a1,0) and f1(a2,0)
    t2 = f2(b1,0) and f2(b2,0)
    t3 = f3(c1,0) and f3(c2,0)
    return t1 and t2 and t3
    
def ge(a1,a2):
    return a1 >= a2

def le(a1,a2):
    return a1 <= a2

# Checks if 2x2x3 contains a 2x2x2 with nonneg 4
def check_r4(t):
    a1 = not check_r3(tl.tensor([t[0],t[1]]))
    a2 = not check_r3(tl.tensor([t[0],t[2]]))
    a3 = not check_r3(tl.tensor([t[1],t[2]]))
    return a1 or a2 or a3 

In [13]:
def mat_inv(t):
    tens = t.copy()
    M_1 = Matrix([Matrix(tens[0][1]).transpose(),Matrix(tens[0][0]).transpose()])
    M_2 = Matrix([Matrix(tens[1][1]).transpose(),Matrix(tens[1][0]).transpose()])
    M_3 = Matrix([Matrix(tens[2][1]).transpose(),Matrix(tens[2][0]).transpose()])
    tens[0] = M_1
    tens[1] = M_2
    tens[2] = M_3
    return tens

def mat_trans(t):
    tens = t.copy()
    tens[0] = tens[0].transpose()
    tens[1] = tens[1].transpose()
    tens[2] = tens[2].transpose()
    return tens

def rotate(t):
    return mat_inv(mat_trans(t))


def proc_tensor(t):
    tens = t.copy()
    M_b = Matrix([[Matrix(tens[:,0,0]), Matrix(tens[:,0,1]), Matrix(tens[:,1,1])]]).transpose()
    M_a = Matrix([[Matrix(tens[:,0,0]), Matrix(tens[:,0,1]), Matrix(tens[:,1,0])]]).transpose()
    if abs(M_a.det()) < 10:
        print("warning det too small")
    a = 1
    b = M_b.det() / M_a.det()
    M = Matrix([[tens[0][0][0],0,a,0,0],[0,tens[0][0][1],b,0,0],
                [tens[1][0][0],0,0,a,0],[0,tens[1][0][1],0,b,0],
                [tens[2][0][0],0,0,0,a],[0,tens[2][0][1],0,0,b]])
    R = Matrix([Matrix(tens[0,1]),Matrix(tens[1,1]),Matrix(tens[2,1])])
    M_sub = Matrix(M[0:5,0:5])
    #sol = np.array(np.array(M_sub.inv(), dtype = "float") @ np.array(Matrix(R[0:5]), dtype = "float"), dtype = "float")
    sol = la.solve(np.array(M.T@M,dtype = "float"), np.array(M.T@R, dtype = "float"))
    return np.append(np.array([b], dtype = "float"),sol)


def test(tens):
    ret = [0] *11
    c1 = proc_tensor(tens)
    if np.all(c1>=0):
        ret[1] = 1
    c2 = proc_tensor(mat_trans(tens))
    if np.all(c2>=0):
        ret[2] = 1
    c3 = proc_tensor(mat_inv(tens))
    if np.all(c3>=0):
        ret[3] = 1
    c4 = proc_tensor(rotate(tens))
    if np.all(c4>=0):
        ret[4] = 1
    c5 = proc_tensor(mat_trans(mat_inv(tens)))
    if np.all(c5>=0):
        ret[5] = 1
    total = sum(ret)
    if total >= 1:
        ret[0] = 1
    ret[6+total] = 1
    r4 = check_r4(tens)
    if r4:
        ret[6] = 1
    else:
        ret[6] = 0
    return ret

# happens w.p 0 so probably pointless to try
# also probably implied by the larger guess
def proc_small(tens):
    M = Matrix([[Matrix(tens[:,0,0]), Matrix(tens[:,1,1]),Matrix(tens[:,0,1])]])
    M1 = Matrix([[Matrix(tens[:,0,0]), Matrix(tens[:,1,1]),Matrix(tens[:,1,0])]])
    M2 = Matrix([[Matrix(tens[:,0,0]), Matrix(tens[:,1,0]),Matrix(tens[:,0,1])]])
    M3 = Matrix([[Matrix(tens[:,1,1]), Matrix(tens[:,1,0]),Matrix(tens[:,0,1])]])
    return solve_mat(M) or solve_mat(M1) or solve_mat(M2) or solve_mat(M3)

def solve_mat(M):
    if abs(det(M))> 100:
        return False
    M1 = M[:,0:2]
    R = M[:,2]
    sol = la.solve(np.array(M1.T@M1,dtype = "float"),np.array(M1.T@R, dtype = "float"))
    return np.all(sol >=0)
    

In [5]:
def loop_rotations(i):
    tens = rand_tensor()
    tens = rank_tree()
    ret = [0] *11
    c1 = proc_tensor(tens)
    if np.all(c1>=0):
        ret[1] = 1
    c2 = proc_tensor(mat_trans(tens))
    if np.all(c2>=0):
        ret[2] = 1
    c3 = proc_tensor(mat_inv(tens))
    if np.all(c3>=0):
        ret[3] = 1
    c4 = proc_tensor(rotate(tens))
    if np.all(c4>=0):
        ret[4] = 1
    c5 = proc_small(tens)
    if c5:
        ret[5] = 1
    total = sum(ret)
    if total >= 1:
        ret[0] = 1
    #ret[6+total] = 1
    r4 = check_r4(tens)
    if r4:
        ret[6] = 1
    else:
        ret[6] = 0
    return ret

In [114]:
te = time.time()
total = 1000
results = Parallel(n_jobs=6)(delayed(loop_rotations)(i) for i in range(total))
res = [sum(x) for x in zip(*results)]
print(res)
print(time.time() - te)

[772, 359, 327, 300, 326, 0, 0, 0, 0, 0, 0]
3.8584988117218018


In [7]:
tens = rank_tree()
#tens = tl.tensor(np.random.randint(1, 100000, size=(3,2,2)))*1.0
M_b = Matrix([[Matrix(tens[:,0,0]), Matrix(tens[:,0,1]), Matrix(tens[:,1,1])]]).transpose()
M_a = Matrix([[Matrix(tens[:,0,0]), Matrix(tens[:,0,1]), Matrix(tens[:,1,0])]]).transpose()
a = 1
b = M_b.det() / M_a.det()
M = Matrix([[tens[0][0][0],0,a,0,0],[0,tens[0][0][1],b,0,0],
            [tens[1][0][0],0,0,a,0],[0,tens[1][0][1],0,b,0],
            [tens[2][0][0],0,0,0,a],[0,tens[2][0][1],0,0,b]])
R = Matrix([Matrix(tens[0,1]),Matrix(tens[1,1]),Matrix(tens[2,1])])
M_sub = Matrix(M[0:5,0:5])
sol = np.array((M_sub.adjugate() @ Matrix(R[0:5])) /M_sub.det(), dtype = "float")
sol1 = la.solve(np.array(M.T @ M, dtype = "float"), np.array(M.T @ R, dtype = "float"))
sol


array([[ 1.11433914e+01],
       [ 1.25368265e+01],
       [-1.83211490e+07],
       [-2.00294909e+07],
       [-1.72537657e+06]])

In [16]:
tens = rank_tree()
proc_tensor(tens)

array([ 2.10764179e+00,  1.25438573e+00,  1.22806933e+00, -3.02891638e+06,
       -1.00037285e+07, -5.20657049e+06])

In [22]:
for i in range(1000):
    tens = rank_tree()
    proc_tensor(tens)
    proc_tensor(mat_inv(tens))
    proc_tensor(mat_trans(tens))
    proc_tensor(rotate(tens))


In [12]:
t

array([[[71639., 57940.],
        [82820., 77230.]],

       [[59299., 85536.],
        [62629., 50797.]],

       [[56222.,  1604.],
        [85405., 46788.]]])

In [14]:
sol = proc_tensor(tens)
sol

array([-1.44926912e-01,  7.33860205e+00,  2.74634528e-01, -3.35925544e+05,
       -4.29858967e+05, -6.28423882e+05])

In [69]:
sol = proc_tensor(tens)
b = sol[0]
x = sol[1]
y = sol[2]
z1 = sol[3]
z2 = sol[4]
z3 = sol[5]
t = tens
a1=t[0,0,0]*x + z1 - t[0,1,0]
a2=t[0,0,1]*y + b*z1 - t[0,1,1]
a3=t[1,0,0]*x + z2 - t[1,1,0]
a4=t[1,0,1]*y + b*z2 - t[1,1,1]
a5=t[2,0,0]*x + z3 - t[2,1,0]
a6=t[2,0,1]*y + b*z3 - t[2,1,1]
print(max(a1,a2,a3,a4,a5,a6))

1.8189894035458565e-12


In [39]:
mat_inv(tens)

array([[[ 7477., 75400.],
        [46794., 97276.]],

       [[19990., 85580.],
        [61299., 84774.]],

       [[ 9807., 99704.],
        [86969., 31418.]]])

In [40]:
sol = proc_tensor(mat_inv(tens))
t = tens
b = sol[0]
x = sol[1]
y = sol[2]
z1 = sol[3]
z2 = sol[4]
z3 = sol[5]
a1=t[0,1,0]*x + z1 - t[0,0,0]
a2=t[0,1,1]*y + b*z1 - t[0,0,1]
a3=t[1,1,0]*x + z2 - t[1,0,0]
a4=t[1,1,1]*y + b*z2 - t[1,0,1]
a5=t[2,1,0]*x + z3 - t[2,0,0]
a6=t[2,1,1]*y + b*z3 - t[2,0,1]
print(max(a1,a2,a3,a4,a5,a6))

1.4551915228366852e-10


In [45]:
count = [0,0,0]
for i in range(100):
    tens = rank_tree()
    c1 = np.all(proc_tensor(tens) >=0)
    c2 = np.all(proc_tensor(mat_inv(tens))>=0)
    if c1 and c2:
        count[0] += 1
    if c1:
        count[1] += 1
    if c2:
        count[2] += 1
print(count)

[0, 37, 40]


In [82]:
sol = proc_tensor(rotate(tens))
b = sol[0]
x = sol[1]
y = sol[2]
z1 = sol[3]
z2 = sol[4]
z3 = sol[5]
t = tens
a1=t[0,0,1]*x + z1 - t[0,0,0]
a2=t[0,1,1]*y + b*z1 - t[0,1,0]
a3=t[1,0,1]*x + z2 - t[1,0,0]
a4=t[1,1,1]*y + b*z2 - t[1,1,0]
a5=t[2,0,1]*x + z3 - t[2,0,0]
a6=t[2,1,1]*y + b*z3 - t[2,1,0]
print(max(a1,a2,a3,a4,a5,a6))

7.275957614183426e-12


In [83]:
tens

array([[[15441., 12988.],
        [ 7623.,  3708.]],

       [[13023., 21032.],
        [ 5337.,  6444.]],

       [[ 5309., 12780.],
        [ 1827.,  4176.]]])

In [84]:
tens[:,1,1]

array([3708., 6444., 4176.])

In [111]:
a1 = np.array([1,0])
a2 = np.array([x,1])
a3 = np.array(tens[:,0,1])
t1= np.kron(np.kron(a3,a1),a2).reshape(3,2,2)

b1 = np.array([0,1])
b2 = np.array([y,1])
b3 = np.array(tens[:,1,1])
t2 = np.kron(np.kron(b3,b1),b2).reshape(3,2,2)

c1 = np.array([1,b])
c2 = np.array([1,0])
c3 = np.array([z1,z2,z3])
t3 = np.kron(np.kron(c3,c1),c2).reshape(3,2,2)
t3+t1+t2-tens

array([[[z1 - 4917.64910788592, 0],
        [-2.42325425351385*z1 - 6230.85827512589, 0]],

       [[z2 - 3192.18718106464, 0],
        [-2.42325425351385*z2 - 3411.14404111334, 0]],

       [[z3 - 4296.51586080836, 0],
        [-2.42325425351385*z3 - 4493.63023415741, 0]]], dtype=object)

In [6]:
proc_tensor(rotate(rotate(tens)))

NameError: name 'tens' is not defined

In [90]:
proc_tensor(mat_trans(tens))

array([ 4.95783144e-01,  1.88814841e+00,  1.53788225e+00, -1.61668997e+04,
       -3.55735681e+03,  2.75582006e+03])

In [91]:
proc_tensor(mat_inv(tens))

array([1.88814841e+00, 1.64283973e+00, 2.01701089e+00, 2.91763274e+03,
       4.25516436e+03, 2.30753181e+03])

In [92]:
rotate(rotate(tens))

array([[[ 3708.,  7623.],
        [12988., 15441.]],

       [[ 6444.,  5337.],
        [21032., 13023.]],

       [[ 4176.,  1827.],
        [12780.,  5309.]]])

In [44]:
tens = rand_tensor()
tens

array([[[36161., 80559.],
        [84230., 61531.]],

       [[ 5502., 47097.],
        [31617., 36592.]],

       [[94529., 38236.],
        [52133., 45258.]]])

In [50]:
M = Matrix([[Matrix(tens[:,0,0]), Matrix(tens[:,1,1]),Matrix(tens[:,1,0])]])
M

Matrix([
[36161.0, 61531.0, 84230.0],
[ 5502.0, 36592.0, 31617.0],
[94529.0, 45258.0, 52133.0]])

In [67]:
counter = 0
tensors = []
for i in range(10000):
    tens = rank_tree()
    M = Matrix([[Matrix(tens[:,0,0]), Matrix(tens[:,1,1]),Matrix(tens[:,0,1])]])
    M1 = Matrix([[Matrix(tens[:,0,0]), Matrix(tens[:,1,1]),Matrix(tens[:,1,0])]])
    M2 = Matrix([[Matrix(tens[:,0,0]), Matrix(tens[:,1,0]),Matrix(tens[:,0,1])]])
    M3 = Matrix([[Matrix(tens[:,1,1]), Matrix(tens[:,1,0]),Matrix(tens[:,0,1])]])
    if abs(det(M)) < 10 or abs(det(M1))<10 or abs(det(M2))<10 or abs(det(M3))<10:
        counter += 1
        tensors.append(tens)
print(counter)
        

11


In [70]:
tens = tensors[0]
tens

array([[[ 5140.,  5690.],
        [11720., 12375.]],

       [[ 3356.,  4192.],
        [ 7472.,  9155.]],

       [[ 4520.,  5719.],
        [10036., 12495.]]])

In [88]:
M = Matrix([[Matrix(tens[:,0,0]), Matrix(tens[:,1,1]),Matrix(tens[:,0,1])]])
M

Matrix([
[5140.0, 12375.0, 5690.0],
[3356.0,  9155.0, 4192.0],
[4520.0, 12495.0, 5719.0]])

In [89]:
M[:,2]

Matrix([
[5690.0],
[4192.0],
[5719.0]])

In [90]:
M1 = M[:,0:2]
R = M[:,2]
sol = la.solve(np.array(M1.T@M1,dtype = "float"),np.array(M1.T@R, dtype = "float"))
sol

array([[0.03907749],
       [0.44356701]])

In [79]:
x = sol[0][0]
y = sol[1][0]
a1 = np.array([1,0])
a2 = np.array([1,x])
a3 = np.array(tens[:,0,0], dtype = "float")
t1= np.kron(np.kron(a3,a1),a2).reshape(3,2,2)

b1 = np.array([y,1])
b2 = np.array([0,1])
b3 = np.array(tens[:,1,1], dtype = "float")
t2 = np.kron(np.kron(b3,b1),b2).reshape(3,2,2)

c1 = np.array([0,1])
c2 = np.array([1,0])
c3 = np.array(tens[:,1,0], dtype = "float")
t3 = np.kron(np.kron(c3,c1),c2).reshape(3,2,2)
t3+t1+t2-tens

array([[[ 0.00000000e+00,  1.00044417e-11],
        [ 0.00000000e+00,  0.00000000e+00]],

       [[ 0.00000000e+00, -4.54747351e-12],
        [ 0.00000000e+00,  0.00000000e+00]],

       [[ 0.00000000e+00, -7.27595761e-12],
        [ 0.00000000e+00,  0.00000000e+00]]])

In [81]:
np.all(sol>=0)

True